In [55]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, random_split
from torchvision import transforms

In [71]:
# 1. Configuration and Data Loading
BATCH_SIZE = 4
transform = transforms.Compose([
    transforms.ToTensor(),
    # transforms.Normalize((0.1307,), (0.3081,))
])

In [72]:
df = pd.read_csv("student_learning_trajectory.csv")

In [73]:
df.head(5)

,student_id,week,study_hours,sleep_hours,stress_level,attendance_rate,screen_time_hours,caffeine_intake,learning_efficiency,fatigue_index,quiz_score,assignment_score,performance_index
0,0,1,7.888120,7.318902,5.558083,0.951052,3.838244,1,0.425517,0.555808,63.270200,109.638265,91.091039
1,0,2,9.950370,5.987169,5.628495,0.759198,2.175393,2,0.440707,0.900460,79.333452,113.540690,99.857795
2,0,3,7.849006,7.375698,3.798723,0.820831,3.796587,0,0.453817,0.379872,77.072262,112.927018,98.585116
3,0,4,8.994472,7.822545,2.558313,0.870886,1.080660,1,0.468138,0.255831,74.500142,119.813462,101.688134
4,0,5,7.990915,5.416706,6.547401,0.796186,2.306644,1,0.481408,1.182505,57.751540,117.716967,93.730796


In [74]:
df.columns

Index(['student_id', 'week', 'study_hours', 'sleep_hours', 'stress_level',
       'attendance_rate', 'screen_time_hours', 'caffeine_intake',
       'learning_efficiency', 'fatigue_index', 'quiz_score',
       'assignment_score', 'performance_index'],
      dtype='object')

In [75]:
pivot_result = pd.pivot_table(
    df,
    values=["study_hours", "sleep_hours", "stress_level", "attendance_rate", "screen_time_hours",
            "caffeine_intake", "learning_efficiency", "fatigue_index", "quiz_score", "assignment_score",
            "performance_index"],
    index=["student_id", "week"],
    aggfunc="mean",
    fill_value=0
)

In [76]:
x_col_names = ["study_hours", "sleep_hours", "stress_level", "attendance_rate", "screen_time_hours",
               "caffeine_intake", "learning_efficiency", "fatigue_index", "quiz_score", "assignment_score"]
y_col_names = "performance_index"

In [78]:
pivot_result.loc[0, x_col_names]

,study_hours,sleep_hours,stress_level,attendance_rate,screen_time_hours,caffeine_intake,learning_efficiency,fatigue_index,quiz_score,assignment_score
week,,,,,,,,,,
1,7.888120,7.318902,5.558083,0.951052,3.838244,1.0,0.425517,0.555808,63.270200,109.638265
2,9.950370,5.987169,5.628495,0.759198,2.175393,2.0,0.440707,0.900460,79.333452,113.540690
3,7.849006,7.375698,3.798723,0.820831,3.796587,0.0,0.453817,0.379872,77.072262,112.927018
4,8.994472,7.822545,2.558313,0.870886,1.080660,1.0,0.468138,0.255831,74.500142,119.813462
5,7.990915,5.416706,6.547401,0.796186,2.306644,1.0,0.481408,1.182505,57.751540,117.716967
6,9.887748,6.160782,4.381575,0.883126,6.951090,1.0,0.496545,0.717897,63.487673,114.420171
7,9.834795,7.296561,2.924340,0.842420,6.945927,3.0,0.511636,0.292434,76.623424,116.342893
8,8.939136,7.087047,4.401985,0.859176,1.024862,2.0,0.525903,0.440199,66.458878,123.416689
9,8.191506,6.498243,6.830804,0.882875,3.940480,4.0,0.539395,0.850333,68.838832,120.174227


In [90]:
data_input = [
    (pivot_result.loc[i, x_col_names].to_numpy(dtype=np.float32),
    pivot_result.loc[i, y_col_names].to_numpy(dtype=np.float32))
    for i in pivot_result.index.levels[0]
]

In [91]:
train_dataset, test_dataset = random_split(data_input, [0.7, 0.3])

In [92]:
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=1000, shuffle=False)

In [93]:
# Use GPU if available, otherwise CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [102]:
# 2. Define the Neural Network Model
class SimpleNN(nn.Module):
    def __init__(self):
        super(SimpleNN, self).__init__()
        # Input layer (28*28 = 784 pixels) to a hidden layer of 32 neurons
        self.fc1 = nn.Linear(16 * 10, 32)
        self.fc2 = nn.Linear(32, 16)

    def forward(self, x):
        # Flatten the input image from 28x28 to a 784-element vector
        x = x.view(-1, 16 * 10)
        x = F.tanh(self.fc1(x))
        x = self.fc2(x)
        # Apply log softmax for the final output (often used with NLLLoss, which is part of CrossEntropyLoss)
        return F.log_softmax(x, dim=1)

In [103]:
model = SimpleNN().to(device)

In [104]:
# 3. Define Loss and Optimizer
criterion = nn.NLLLoss()  # Negative Log-Likelihood Loss (suitable for log_softmax output)
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [109]:
# 4. Training Loop
NUM_EPOCHS = 5

for epoch in range(NUM_EPOCHS):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)

        optimizer.zero_grad()  # Clear gradients
        output = model(data)  # Forward pass
        loss = criterion(output, target)  # Calculate loss
        loss.backward()  # Backward pass (calculate gradients)
        optimizer.step()  # Update weights

        if batch_idx % 100 == 0:
            print(f'Train Epoch: {epoch + 1}/{NUM_EPOCHS} [{batch_idx * len(data)}/{len(train_loader.dataset)} '
                  f'({100. * batch_idx / len(train_loader):.0f}%)]\tLoss: {loss.item():.6f}')

torch.Size([4, 16])
torch.Size([4, 16])


RuntimeError: 0D or 1D target tensor expected, multi-target not supported

In [ ]:
# 5. Testing the Model
model.eval()
test_loss = 0
correct = 0

In [ ]:
with torch.no_grad():
    for data, target in test_loader:
        data, target = data.to(device), target.to(device)
        output = model(data)
        test_loss += F.nll_loss(output, target, reduction='sum').item()
        pred = output.argmax(dim=1, keepdim=True)
        correct += pred.eq(target.view_as(pred)).sum().item()

In [ ]:
test_loss /= len(test_loader.dataset)
accuracy = 100. * correct / len(test_loader.dataset)

print(f'\nTest set: Average loss: {test_loss:.4f}, Accuracy: {correct}/{len(test_loader.dataset)} '
      f'({accuracy:.2f}%)\n')

# 6. Save the model (optional)
torch.save(model.state_dict(), "students_simple_nn.pth")